## Check the setup and connect to the database

In [1]:
%run "../check_setup.ipynb"

SAP HANA Client for Python: 2.23.25010300
Connected to SAP HANA db version 4.00.000.00.1737457831 (fa/CE2024.40) 
at c5889dd5-e0f6-4930-8408-94d53ca61dbf.hna0.prod-us10.hanacloud.ondemand.com:443 as CODEJAMHANAML40
Current time on the SAP HANA server: 2025-01-29 21:25:05.684000


In [2]:
myconn.get_tables(schema='NHTSA')

,TABLE_NAME
0,COMPLAINTS


# Car complaints data

The __National Highway Traffic Safety Administration (NHTSA)__, is part of the U.S. Department of Transportation.
Complaint information entered into NHTSA’s Office of Defects Investigation __vehicle owner's complaint database__ is used with other data sources to identify __safety issues__ that warrant investigation and to determine if a safety-related defect trend exists. Complaint information is also analyzed to monitor existing recalls for proper scope and adequacy. The NHTSA provides a large dataset of complaints related to cars in the US:[https://www.nhtsa.gov/nhtsa-datasets-and-apis#complaints].
For this demo scenario, we've loaded the 2024 complaints data, for detail instructions see appendix.


In [4]:
# Create a HANA dataframe for the complaints data, pre-loaded into SAP HANA Cloud
hdf_complaints=myconn.table('COMPLAINTS', schema='NHTSA')
# hdf_complaints.get_table_structure()

In [5]:
# Overview the complaints data
import pandas as pd
pd.set_option('max_colwidth', None) 
display(
    hdf_complaints.filter("""PROD_TYPE='V'""") # Vehicle-related complaint
    .select('CMPLID', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'COMPDESC', 'CDESCR')
    .head(1).collect().T
)

,0
CMPLID,1639185
MAKETXT,FORD
MODELTXT,ESCAPE
YEARTXT,2020
COMPDESC,ENGINE
CDESCR,"SINCE DAY ONE OF OWNERSHIP, IT HAS A VIBRATION IN THE STEERING WHEN APPROACHING 25MPH THEN THE VIBRATION CONTINUES ALL THE WAY TO 75MPH. ALSO AT APPROX. 25MPH AN LOW HUMMING SOUND STARTS. THE HUMMING SOUND GETS LOADER AS SPEED INCREASES ALONG WITH THE ROAD NOISE. THE HUMMING AND VIBRATION STOP AS SOON AS YOU TAKE YOUR FOOT OFF THE ACCELERATOR. BUT RESUMES AGAIN AS SOON AS THE ACCELERATOR IS PRESSED. THIS VIBRATION FEELS AS IF A WHEEL IS LOOSE OR COMING OFF OR THE THE ENGINE AND TRANSMISSION ARE GOING TO FAIL CAUSING A SERIOUS PROBLEM. THIS BY NO MEANS IS NORMAL. BOTH DEALERSHIPS PERSONNEL THAT DROVE THE VEHICAL SAID ITS NOT NORMAL."


In [6]:
# Let's filter on specific component-groups, for detailed classification analysis
hdf_carcomplaints=(hdf_complaints
    .select('CMPLID', 'MFR_NAME', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'FUEL_TYPE', 'CRASH', 'FIRE', 'STATE', 'CMPL_TYPE',
            'ANTI_BRAKES_YN', 'CRUISE_CONT_YN', 'DRIVE_TRAIN', 'VEHICLES_TOWED_YN', 'CDESCR', 'COMPDESC')
    .filter('''COMPDESC IN ('AIR BAGS','ELECTRICAL SYSTEM', 'SERVICE BRAKES','STEERING')'''))

hdf_carcomplaints.count()

122447

In [7]:
pd.set_option('max_colwidth', None) 
display(hdf_carcomplaints.head(1).collect().T)

,0
CMPLID,1766833
MFR_NAME,Ford Motor Company
MAKETXT,FORD
MODELTXT,ESCAPE
YEARTXT,2009
FUEL_TYPE,None
CRASH,N
FIRE,N
STATE,MI
CMPL_TYPE,IVOQ


## Text splitting, preparing complaints description text for vectorization

Text embedding models and Large Language Models (LLMs) often have token length limits, hence managing the text length before running it through such models is a frequent preprocessing task. For that purpose, a new text splitting function __hana_ml.text.text_splitter__ is being introduced and explained in more detail in the following blog post: [Text chunking - an exciting new NLP function in SAP HANA Cloud](https://community.sap.com/t5/technology-blogs-by-sap/text-chunking-an-exciting-new-nlp-function-in-sap-hana-cloud/ba-p/13958766).

In [8]:
# Determining the character length (using SQL length-fct) of a given text. For western languages, character-length / 3 or 4 is giving an approximate token length
# Note, the text analysis function applied above, also determines the token length specifically

hdf_carcomplaints.select('CMPLID', ('LENGTH("CDESCR")', 'LEN_CDESCR')).sort('LEN_CDESCR', desc=True).head(100).collect()

,CMPLID,LEN_CDESCR
0,1972918,2048
1,2030207,2048
2,1982717,2048
3,1954985,2048
4,1945086,2048
...,...,...
95,1996080,2048
96,2001261,2048
97,1979041,2048
98,2024784,2048


In [9]:
hdf_carcomplaints.filter("""CMPLID=1918524""").select("CDESCR").collect().T

,0
CDESCR,"My name is [XXX] and I have been a Nissan Consumer for over 10 years, and this is the second time I have had major issues with my breaks. Unfortunately, I did not send an email when the 1st occurrence happened but, in my brand, new Rouge around 2017, my front and back breaks went completely out and had to be replaced at 6 months, but I continued to patronize your company. This occurrence I am sending an email regarding my 2022 Nissan Pathfinder I purchased from Passport Nissan, Marlowe Heights. I called the consumer customer service number on 8/12 and was provided case # [XXX}. Nissan Consumer line says, oh, there is nothing we can do because your vehicle is not under warranty and there has not been a recall! I brought my truck in for an oil change and tire rotation (which is part of the extended warranty when I purchased my truck) on July 11, 2003, and was not informed that my brakes needed any attention. The week of 8/7/2023 I noticed my brakes were really loud when I went in reverse so I looked up when should a car/truck need new brakes seeing as though I have only had this vehicle for just a year in July. It should brake should not be needed for at least 30-60,000 miles, here I am at 12, 483 miles. I called the dealership and was informed this is likely wear and tear and that I would be responsible for the cost of new breaks if needed. I am baffled because the point of me leasing a new vehicle is an attempt to avoid major costs such as brake and engine concerns before my 3-year lease is up. Yes, I leased 3 years prior to this lease a year ago! I make an appointment and take my truck to Nissan of Marlow Heights and informed the worker of my concern and he immediately tells me, oh, it’s just a noise your bakes make early in the morning, my Toyota does the same thing in the mornings. I explained to him that it’s not just in the morning but when I go in reverse, and I demonstrated this concern and he was like, oh no that is not norm INFORMATION REDACTED PURSUANT TO THE FREEDOM OF INFORMATION ACT (FOIA), 5 U."


In [15]:
# Applying the Text Splitter with recursive-splitting, available with hana-ml 2.23
from hana_ml.text.text_splitter import TextSplitter

splitter = TextSplitter(split_type='recursive', chunk_size=512, overlap=64)
#splitter._extend_pal_parameter({'GLOBAL_SEPARATOR':'[.]', 'KEEP_SEPARATOR':1})
splitted_text = splitter.split_text(
    hdf_carcomplaints.filter("LENGTH(CDESCR)>=512").select('CMPLID', 'CDESCR').head(10), 
    order_status=True
    )
#print(splitted_text.shape)
display(splitter.statistics_.collect())

display(splitted_text.select("*", ('LENGTH("CONTENT")', "CHUNCK_SIZE")).head(15).collect())

,STAT_NAME,STAT_VALUE
0,GLOBAL_SEPARATOR_LIST,"{""Separator"":""[\n\n,\n, ]""}"


,CMPLID,SUB_ID,CONTENT,CHUNCK_SIZE
0,1766833,0,"Subject vehicle was a 2009 Ford Escape with 74,123 miles. Driving the vehicle the front passenger lower control arm separated from the k-frame/cradle causing the vehicle to lose control. Loss of steering was experienced as well as tire lock up from the front tire lodging against the back of the fender. The CV shaft also separated during the failure resulting in loss of transmission engagement. There was no warning prior to the event, the failure was sudden and immediate. The loss of vehicle control was",508
1,1766833,1,"was sudden and immediate. The loss of vehicle control was extreme. Failure is identical to RCRIT-14V165-9596 for the 01-04 Escapes. The reason the recall was only isolated to the 01-04 Escapes is unknown. In my opinion, whatever PCA actions or supplier related issues identified by Ford Motor Company during the prior recall investigation to support the recall only affecting 01-04 model years was either incomplete or inaccurate. The frame rust through condition extends beyond the model years listed in the",511
2,1766833,2,through condition extends beyond the model years listed in the recall and was not resolved as shown by subsequent failures on vehicles outside of the recall model year window. No mechanism was put in place to prevent the control arm separation in the later model year Escapes to prevent the same safety issue from occurring. I was unable to obtain a front cradle from a local salvage yard that did not exhibit perforation present or starting supporting this condition is occurring on all model year Escapes from,512
3,1766833,3,this condition is occurring on all model year Escapes from 2001 to 2012 in corrosion prone areas. The recall should be extended to all model years involved to prevent potential accidents.,188
4,1766844,4,"Subject vehicle is a 2009 Ford Escape with 74,123 miles. Driving the vehicle the lower control arm separated from the front frame otherwise known as the engine cradle or subframe. The failure is identical to the subframe failures on the 2001 - 2004 Ford Escapes currently under recall. The failure was immediate with no prior warning. The vehicle exhibited loss of steering control, wheel lock up from the front tire contacting the back of the wheel opening, and loss of transmission engagement from cv joint",510
5,1766844,5,"wheel opening, and loss of transmission engagement from cv joint separation. In my opinion, whatever PCA actions or quality root cause was identified by Ford Motor Company during the prior recall investigation isolating the condition to 2001 - 2004 Escapes was either incomplete or inaccurate. This is supported by the same failure occurring in Escapes outside of that model year window. No mechanism was added after the 2004 Escapes were produced to prevent this failure from occurring. Images attached show",511
6,1766844,6,to prevent this failure from occurring. Images attached show the failure that was experienced. I was unable to obtain a good used frame locally that did not exhibit signs of corrosion with perforation present or starting. This further supports the failure mode is beyond the scope of the initial recall. The recall should be expanded to prevent future accidents that could have the potential for injury or death.,415
7,1900521,7,"I have been the only driver/owner since May of 2020. Vehicle has not been in any type of accident. 2020 Ford Escape has approximately 18,000 miles. I did purchase (bumper to bumper) warranty. Vehicle is scheduled for service on July 10, 2023. Ford dealerships are understaffed and can only service vehicles on urgency of issue or issues. July date was soonest and was from the 3rd Ford dealership I called. Issues started on May 13, 2023. While vehicle is parked and while it’s being driven, the following",507
8,1900521,8,"vehicle is parked and while it’s being driven, the following warnings occur: •AWD Off •AWD Service Required •Powertrain Malfunction •Powertrain Service Required

## Generating Text Embeddings in SAP HANA Cloud

SAP HANA Cloud introduces availability of a text embedding model, targeted for vectorization of text data already stored within the database. It is specifically useful for vector engine similarity search scenarios or machine learning tasks, implicitly making use text embedding vectors unlocking the semantic understanding of text data for the analysis. A detailed capability introduction can be found in the following blog post [Text Embedding Service in SAP HANA Cloud Predictive Analysis Library (PAL)](https://community.sap.com/t5/enterprise-resource-planning-blogs-by-sap/text-embedding-service-in-sap-hana-cloud-predictive-analysis-library-pal/ba-p/13958864).

Running vectorizations for 122447 recs took 37 minutes (2244 secs) on 4 vCPUs.

For the sake of the exercise let's reduce the number of records by limiting only to cars with hubrid engine.

In [14]:
print(f"""Number of records selected for further processing: {(hdf_carcomplaints_he := hdf_carcomplaints.filter(''' "FUEL_TYPE"='HE' ''')).count()}""")

Number of records selected for further processing: 906


In [12]:
### Generating Text Embeddings in SAP HANA Cloud with the new PAL function, function available with hana-ml 2.23.
from hana_ml.text.pal_embeddings import PALEmbeddings
pe = PALEmbeddings()
textembeddings = pe.fit_transform(hdf_carcomplaints_he, key="CMPLID", target=["CDESCR"], thread_number=10, batch_size=10) #, max_token_num=512
print(f"{textembeddings.count()} records processed in {round(pe.runtime, 3)} sec")

906 records processed in 23.572 sec


In [13]:
textembeddings.get_table_structure()

{'CMPLID': 'INT',
 'MFR_NAME': 'NVARCHAR(5000)',
 'MAKETXT': 'NVARCHAR(5000)',
 'MODELTXT': 'NVARCHAR(5000)',
 'YEARTXT': 'INT',
 'FUEL_TYPE': 'NVARCHAR(5000)',
 'CRASH': 'NVARCHAR(5000)',
 'FIRE': 'NVARCHAR(5000)',
 'STATE': 'NVARCHAR(5000)',
 'CMPL_TYPE': 'NVARCHAR(5000)',
 'ANTI_BRAKES_YN': 'NVARCHAR(5000)',
 'CRUISE_CONT_YN': 'NVARCHAR(5000)',
 'DRIVE_TRAIN': 'NVARCHAR(5000)',
 'VEHICLES_TOWED_YN': 'NVARCHAR(5000)',
 'CDESCR': 'NVARCHAR(5000)',
 'COMPDESC': 'NVARCHAR(5000)',
 'VECTOR_COL_CDESCR': 'REAL_VECTOR'}

In [14]:
# Review the generated Text Embeddings, note they are of vector dimensionality 768 for the current HANA embedding model
cmpl_textembeddings=textembeddings.rename_columns({'VECTOR_COL_CDESCR': 'HANACLOUD_TEXT_EMBEDDING'}).to_tail('COMPDESC')
display(cmpl_textembeddings.head(1).collect().T)

,0
CMPLID,1673124
MFR_NAME,"Tesla, Inc."
MAKETXT,TESLA
MODELTXT,MODEL S
YEARTXT,2015
FUEL_TYPE,HE
CRASH,N
FIRE,N
STATE,OH
CMPL_TYPE,IVOQ


## Text Embeddings for Advanced Text Mining using ANNS

New text mining techniques can now be applied leveraging existing text embedding vectors (or create them implicitly from the original text) unlocking the semantic understanding of text data for text mining tasks like text document classification. Thus elevating a classic task from linguistic text understanding to understanding the real natural language context of documents for greatly improved results. This new technique in SAP HANA Cloud Predictive Analysis Library for advanced text mining builds on the concept of approximate neares neighbor search, thus also adressing larger scale scenarios. The advanced text mining function is explained with more detail in the following blog post [New information retrieval techniques in SAP HANA Cloud using BM25 and ANNS for Advanced Text Mining](https://community.sap.com/t5/technology-blogs-by-sap/new-information-retrieval-techniques-in-sap-hana-cloud-using-bm25-and-anns/ba-p/13958729).

In [50]:
hdf_carcomplaints.filter('''MFR_NAME LIKE ('Tesla%')''').agg([('count', 'YEARTXT', 'COUNT')], group_by='YEARTXT').sort(cols='YEARTXT', desc=False).collect()

,YEARTXT,COUNT
0,2012,35
1,2013,173
2,2014,128
3,2015,278
4,2016,296
5,2017,263
6,2018,431
7,2019,251
8,2020,279
9,2021,593


In [60]:
# Preparing the input dataframe for ANNS-advanced text mining
# Required input data schema: document-id, text
carcomplaints_anns_hdf=hdf_carcomplaints.filter('''MFR_NAME LIKE ('Tesla%') AND "YEARTXT"='2022' ''').select('CMPLID', 'CDESCR')
display(carcomplaints_anns_hdf.head(1).collect())
display(carcomplaints_anns_hdf.count())

,CMPLID,CDESCR
0,1855227,The lane departure warning goes off frequently and at random times when there is no issue with it at all. The windows are constantly malfunctioning and turning down when there is nothing obstructing the path of the window. The tail lights will go out at random times and will only be notified by other drivers particularly at night making the vehicle unsafe to operate and unreliable.


541

In [89]:
# Building the ANNS-advanced text mining model
# Note, the complaints shall be clustered into 70 groups for the approximate similarity search
from hana_ml.text.anns_model import ANNSModel

anns = ANNSModel(by_doc=True)
anns.fit(data=carcomplaints_anns_hdf, key='CMPLID', target='CDESCR', group_number=70, max_iteration=100, 
         comment='E/HE-cars complaints search model for 2022c')

print(f"Runtime building the ANN-text mining search model：{anns.runtime} s")

Runtime building the ANN-text mining search model：15.17789363861084 s


In [96]:
# Reviewing the implicit embedding generation results from the ANNS-advanced text mining model building
print(anns.embedding_result_.shape)
display(anns.embedding_result_.head(1).collect().T)
print(f"""Vector cardinality: {anns.embedding_result_.head(1).select(("CARDINALITY(VECTOR_COL)","A")).collect()["A"][0]}""")

[541, 3]


,0
CMPLID,1855227
VECTOR_COL,"[-0.007935982197523117, 0.01113341934978962, 0.08002090454101562, -0.019622335210442543, 0.026045864447951317, -0.04998238384723663, 0.06357908993959427, 0.05571097508072853, 0.0011475024512037635, -0.037405144423246384, -0.0463743731379509, 0.029678858816623688, -0.048545073717832565, 0.06302425265312195, -0.008829998783767223, 0.021310392767190933, 0.07625855505466461, 0.025461165234446526, 0.017485782504081726, 0.016561271622776985, 0.01626119203865528, 0.017201999202370644, -0.009417080320417881, -0.00040049044764600694, 0.050985388457775116, -0.048485081642866135, -0.04416587948799133, 0.024241061881184578, -0.07286423444747925, -0.01245453953742981, 0.03455092012882233, 0.03285488858819008, -0.026608087122440338, 0.03573206439614296, 0.046766798943281174, 0.02361396886408329, 0.026054490357637405, -0.058685239404439926, -0.02217235416173935, 0.026794664561748505, -0.04962014779448509, -0.02213200554251671, -0.04548068717122078, -0.027811134234070778, -0.009396367706358433, 0.013616719283163548, -0.0124001894146204, 0.008171217516064644, 0.005413115490227938, -0.07850267738103867, -0.051774587482213974, -0.0025348959024995565, 0.010324607603251934, 0.043108176440000534, 0.024843139573931694, 0.029277516528964043, 0.00701037747785449, -0.043236277997493744, 0.03129766881465912, -0.02678447775542736, 0.03260762244462967, -0.007663113530725241, -0.016161367297172546, 0.01934557780623436, 0.02691933698952198, -0.04194408282637596, 0.02359333448112011, 0.027589701116085052, -0.02321847528219223, -0.018728213384747505, -0.06040827929973602, 0.004357719328254461, -0.002695891074836254, -0.0009930855594575405, -0.018819166347384453, -0.03189989551901817, -0.007927102968096733, 0.010827921330928802, 0.0171408299356699, 0.016388893127441406, 0.011907706968486309, 0.015588587149977684, 0.041099242866039276, 0.06547380238771439, -0.018332840874791145, -0.036351561546325684, -0.033553265035152435, -0.0012480849400162697, 0.013865831308066845, 0.03564371541142464, -0.06280752271413803, -0.02024013362824917, -0.005908945109695196, 0.006911862175911665, -0.03203699737787247, -0.02469511888921261, 0.04634932801127434, 0.0306816678494215, -0.016025898978114128, -0.010643808171153069, ...]"
EXT_MSG,None


Vector cardinality: 768


In [97]:
# The ANNS-advanced text mining model is kept as a stateful in-memory model for fastest search predictions
display(anns.state_.collect().T)

,0
STATE_ID,343A382E58E85E41BEA49E0E2BB46254
MSG,Succeed. Embedding statistics: Required Version: N/A Used Version: SAP_NEB.20240715 Token Number: 82367
COMMENT,E/HE-cars complaints search model for 2022c
CREATED_AT,2025-01-29 13:54:45.0420000
STATUS,NoReload


In [101]:
# Now, search for CDESCR complaints with the search text: '12v battery is empty'
# Prepare the search query dataframe as: Searchquery-Id, text
hdf_anns_query=myconn.sql(
    '''Select '999' as "QID", 'battery exploded' as "SEARCHTEXT" from "DUMMY"
    UNION ALL
    Select '998' as "QID", '12v battery empty' as "SEARCHTEXT" from "DUMMY"'''
    )
hdf_anns_query.collect()

,QID,SEARCHTEXT
0,998,12 v battery empty
1,999,battery exploded


In [102]:
# Execute the adv. text mining / ANN search 
res = anns.predict(data=hdf_anns_query, key='QID', target='SEARCHTEXT', is_query=True, k_nearest_neighbours=3, k_cluster=70) 
print(f"Runtime ：{anns.runtime} s") 
display(res.collect()) 

Runtime ：0.4298243522644043 s


,TEST_QID,K,TRAIN_ID,DISTANCE,CONTENT
0,999,1,1933767,0.596992,
1,999,2,1789323,0.668881,
2,999,3,1789322,0.668881,
3,998,1,1933767,0.672986,
4,998,2,1896357,0.728740,
5,998,3,1956066,0.734092,


In [103]:
# Joining the adv.text mining/ann search results with the original CDESCR-text
anns_top_searchresults=(
    res.rename_columns({'TEST_QID': 'QID'})
    .set_index("TRAIN_ID")
    .join(carcomplaints_anns_hdf.set_index("CMPLID"))
    )
display(anns_top_searchresults.rename_columns({'TRAIN_ID': 'CMPLID'}).sort('DISTANCE').collect())

,QID,K,CMPLID,DISTANCE,CONTENT,CDESCR
0,999,1,1933767,0.596992,,"Our High voltage battery died on the side of a highway, stranding us until a tow arrived at 4am. During this time, the onboard hazard lights were activated, and no other vehicle systems besides the standard interior features were active. The low voltage battery died within 35 minutes of the high voltage battery. After the low voltage battery died, we were left on the side of the highway at 4am with no active hazard lights, the emergency vehicle door openers were required to be used, however, the windows were rendered unusable, so the doors could not be effectively shut without risking damage to the windows. Having zero hazard or external lighting emitting from the vehicle after just 35 minutes at 4am on a dark stretch of highway put us in an extremely dangerous situation, and in our opinion, the hazards should have functioned for at least 2-3 hours. The vehicle has since been repurchased from the manufacturer as a lemon due to other unrelated issues."
1,999,3,1789322,0.668881,,"On January 19th, around 9:35-9:40am, while driving on Bollinger Canyon Road in San Ramon, the car jerked as if someone rear-ended it and then stopped abruptly in the middle of a busy road. The information screen displayed various error messages regarding battery and drive system. The stop was abrupt and we could have been seriously injured in busy traffic. This was a scary experience. The car had 260 miles range in the battery and no prior warnings/ messages indicating any problem. We called 911 and Tesla roadside assistance for help. San Ramon PD showed up and blocked the lane to avoid someone hitting our car. They had the car towed to the nearest Tesla Service Center in Dublin, CA. While waiting for the tow truck, we attempted to restart the car and computer system two times but the car did not go into ""Drive"" position to be moved. The service center is diagnosing the problem and have informed us that it could be a problem with the motor or drive terrain or inverter of the car. All these are major and crucial components on the car for safe and normal operation. Failure of any of these is catastrophic and can be fatal on freeway speeds or local roads if someone is following too closely and the car stops abruptly. We usually have kids in the car and this is a scary situation to imagine. This needs to be investigated in more detail by someone to find out how many card suffer this fate and how many accidents/ fatalities have been caused by something like this."
2,999,2,1789323,0.668881,,"On January 19th, around 9:35-9:40am, while driving on Bollinger Canyon Road in San Ramon, the car jerked as if someone rear-ended it and then stopped abruptly in the middle of a busy road. The information screen displayed various error messages regarding battery and drive system. The stop was abrupt and we could have been seriously injured in busy traffic. This was a scary experience. The car had 260 miles range in the battery and no prior warnings/ messages indicating any problem. We called 911 and Tesla roadside assistance for help. San Ramon PD showed up and blocked the lane to avoid someone hitting our car. They had the car towed to the nearest Tesla Service Center in Dublin, CA. While waiting for the tow truck, we attempted to restart the car and computer system two times but the car did not go into ""Drive"" position to be moved. The service center is diagnosing the problem and have informed us that it could be a problem with the motor or drive terrain or inverter of the car. All these are major and crucial components on the car for safe and normal operation. Failure of any of these is catastrophic and can be fatal on freeway speeds or local roads if someone is following too closely and the car stops abruptly. We usually have kids in the car and this is a scary situation to imagine. This needs to be investigated in more detail by someone to find out how many card suffer this fate and how many acciden

In [3]:
# List the stateful Adv.text mining/ANNS-models on the system
from hana_ml.text.anns_model import list_models
list_models(myconn).collect()

,STATE_ID,DATASET_TABLE_NAME,COMMENT,CREATED_AT,STATUS
0,546AF4517DB3DE45B15EAC7207E47C54,INVALID.INVALID,E/HE-cars complaints search model,2025-01-29 13:04:29.7350000,NoReload
1,F3759D9D7F509B4D920C520D2FB427D2,INVALID.INVALID,E/HE-cars complaints search model for 2022,2025-01-29 13:08:07.9170000,NoReload
2,343A382E58E85E41BEA49E0E2BB46254,INVALID.INVALID,E/HE-cars complaints search model for 2022c,2025-01-29 13:54:45.0420000,NoReload


In [4]:
# Delete last <anns-object>-model instance
anns.delete_model().collect().T

NameError: name 'anns' is not defined

In [5]:
# List the stateful Adv.text mining/ANNS-models on the system, again
annsmodelstodelete=list_models(myconn)
annsmodelstodelete.collect()

,STATE_ID,DATASET_TABLE_NAME,COMMENT,CREATED_AT,STATUS
0,546AF4517DB3DE45B15EAC7207E47C54,INVALID.INVALID,E/HE-cars complaints search model,2025-01-29 13:04:29.7350000,NoReload
1,F3759D9D7F509B4D920C520D2FB427D2,INVALID.INVALID,E/HE-cars complaints search model for 2022,2025-01-29 13:08:07.9170000,NoReload
2,343A382E58E85E41BEA49E0E2BB46254,INVALID.INVALID,E/HE-cars complaints search model for 2022c,2025-01-29 13:54:45.0420000,NoReload
